# IBM Instance Setup

In [2]:
from dotenv import load_dotenv
import os
import jax
import jax.numpy as jnp
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
from utils.nisq import aachen_connectivity, efficient_connectivity_gates
import pennylane as qml
from datasets.bipartites import BipartiteGraphDataset
from datasets.er import ErdosRenyiGraphDataset
import numpy as np

key = jax.random.PRNGKey(42)

# Experiments Poster

In [3]:
NODES = 8
SP = "Sparse"
MD = "Medium"
DS = "Dense"

ER = "ER"
BP = "Bipartite"

In [5]:
ER8NSParams = np.load("results/params/params_8N_ER_Sparse_LR0.04522709518095061_SIGMA0.6587519702447917_INIT0.17944082088572766_NUMLAYERS1.npy")
ER8NMParams = np.load("results/params/params_8N_ER_Medium_LR0.011220685773844122_SIGMA0.6962179567887462_INIT1.810103830663358_NUMLAYERS5.npy")
ER8NDParams = np.load("results/params/params_8N_ER_Dense_LR0.04206675941389984_SIGMA0.7206244967648459_INIT0.5521825859396434_NUMLAYERS1.npy")
BP8NSParams = np.load("results/params/params_8N_Bipartite_Sparse_LR0.008104680980277149_SIGMA0.5060057247547819_INIT0.7277470256132488_MAXWEIGHT3.npy")
BP8NMParams = np.load("results/params/params_8N_Bipartite_Medium_LR0.003551614405932626_SIGMA1.3352557243570238_INIT0.8853914111571922_NUMLAYERS5.npy")
BP8NDParams = np.load("results/params/params_8N_Bipartite_Dense_LR0.0018973825577720538_SIGMA1.3106607895258173_INIT0.1942402847384792_NUMLAYERS3.npy")

## Experiments

In [6]:
QUBITS = NODES * (NODES - 1) // 2
grid_conn = aachen_connectivity()
dev = qml.device("lightning.qubit", 
                    wires=QUBITS, 
                    shots=512)

MW3 = local_gates(QUBITS, 3)
L1G = efficient_connectivity_gates(grid_conn, QUBITS, 1)
L2G = efficient_connectivity_gates(grid_conn, QUBITS, 2)
L3G = efficient_connectivity_gates(grid_conn, QUBITS, 3)
L4G = efficient_connectivity_gates(grid_conn, QUBITS, 4)
L5G = efficient_connectivity_gates(grid_conn, QUBITS, 5)

MWIQP = iqp.IqpSimulator(QUBITS, MW3, device="lightning.qubit")
L1IQP = iqp.IqpSimulator(QUBITS, L1G, device="lightning.qubit")
L2IQP = iqp.IqpSimulator(QUBITS, L2G, device="lightning.qubit")
L3IQP = iqp.IqpSimulator(QUBITS, L3G, device="lightning.qubit")
L4IQP = iqp.IqpSimulator(QUBITS, L4G, device="lightning.qubit")
L5IQP = iqp.IqpSimulator(QUBITS, L5G, device="lightning.qubit")


@qml.qnode(dev)
def MW_Sample(trained_params):
    MWIQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L1Sample(trained_params):
    L1IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L2Sample(trained_params):
    L2IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L3Sample(trained_params):
    L3IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L4Sample(trained_params):
    L4IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L5Sample(trained_params):
    L5IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

In [9]:
#ER8NSRes  = L1Sample(ER8NSParams)
#ER8NMRes  = L5Sample(ER8NMParams)
#ER8NDRes  = L1Sample(ER8NDParams)


#BP8NDRes = L3Sample(BP8NDParams)
#BP8NMRes = L5Sample(BP8NMParams)
BP8NSRes = MW_Sample(BP8NSParams)

In [10]:
#np.save("results/samples/ER8NS_simulated.npy", ER8NSRes)
#np.save("results/samples/ER8NM_simulated.npy", ER8NMRes)
#np.save("results/samples/ER8ND_simulated.npy", ER8NDRes)
np.save("results/samples/BP8NS_simulated_maxweight3.npy", BP8NSRes)
#np.save("results/samples/BP8NM_simulated.npy", BP8NMRes)
#np.save("results/samples/BP8ND_simulated.npy", BP8NDRes)
print("Samples saved successfully.")

Samples saved successfully.
